In [9]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
% matplotlib inline  
from PIL import Image
import numpy as np

import csv
import os
from scipy import ndimage

In [10]:
def generate_language(img_dir, box_dir, lang_dir):
    cnt = 0
    for sub_dir in os.listdir(box_dir):
        cnt = cnt + 1
        if cnt > 1:
            break
        box_files = os.listdir(os.path.join(box_dir, sub_dir))

        if not os.path.exists(os.path.join(lang_dir, sub_dir)):
            os.mkdir(os.path.join(lang_dir, sub_dir))
        print(os.path.join(lang_dir, sub_dir))
        
        for box_file in box_files:
            box_name = os.path.join(box_dir, sub_dir, box_file)
            file_id, _ = box_file.split('.')
            img_file = file_id + '_color.jpg'
            img_name = os.path.join(img_dir, sub_dir, img_file)
            
            box = []
            with open(box_name) as txt_file:
                box_info = txt_file.readlines()
            # print(box_info)
            for row in box_info:
                row = row.strip().split(' ')
                box.append(row)
            # print 'box', box
            box = np.array(box).astype(np.int)        
            # print 'box', box

            im = np.array(Image.open(img_name), dtype=np.uint8)
            im_height, im_width = im.shape[0], im.shape[1]
            # print im_height, im_width
            
            anno = []
            for i in range(box.shape[0]):
                a = box[i, 0:4]
                cls = box[i, 4].astype(np.int)
                center_x, center_y = (a[0] + a[2]) / 2, (a[1] + a[3]) / 2
                # print 'center', center_x, center_y
                # width = a[2] - a[0]
                # height = a[3] - a[1]
                box_area = (a[2] - a[0]) * (a[3] - a[1])
                # print 'box_area', box_area
                box_area_prop = box_area * 1.0 / im_height / im_width
                # print 'box_area_proportion', box_area_prop
                if box_area_prop < 1.0 / 9:
                    word_size = 0
                elif box_area_prop < 1.0 / 4:
                    word_size = 1
                else:
                    word_size = 2
                relative_center_x = center_x * 1.0 / im_width
                relative_center_y = center_y * 1.0 / im_height
                # print relative_center_x, relative_center_y
                if relative_center_x < 1.0 / 3:
                    word_loc_x = 0
                elif relative_center_x < 1.0 * 2 / 3:
                    word_loc_x = 1
                else:
                    word_loc_x = 2
                if relative_center_y < 1.0 / 3:
                    word_loc_y = 0
                elif relative_center_y < 1.0 * 2 / 3:
                    word_loc_y = 1
                else:
                    word_loc_y = 2
                anno.append([a[0], a[1], a[2], a[3], cls, word_size, word_loc_x, word_loc_y])
            # print anno
            
            if len(anno) == 0:
                continue
            
            lang_file = os.path.join(lang_dir, sub_dir, file_id + '.txt')
            # print(lang_file)
            with open(lang_file, 'w') as handle:
                for i in range(len(anno)):
                    handle.write('%d %d %d %d %d %d %d %d\n' % (anno[i][0], anno[i][1], anno[i][2], anno[i][3], 
                                                                anno[i][4], anno[i][5], anno[i][6], anno[i][7]))

In [11]:
img_dir = '/media/yi/DATA/data-orig/MLT/image'
box_dir = '/home/yi/code/few_shot/mlt/box'
lang_dir = '/home/yi/code/few_shot/mlt_attn/box_lang'

generate_language(img_dir, box_dir, lang_dir)

/home/yi/code/few_shot/mlt_attn/box_lang/b626b9709289bde3a6b3504b7f5586f4
